<a href="https://colab.research.google.com/github/ratulb/pytorch/blob/main/mnist_in_pythor_from_scratch_cleaned_wip3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
import torchvision.transforms as transforms
import random

# Define a transform to convert the data to tensor
transform = transforms.Compose([
    transforms.ToTensor()
])




In [2]:
# Download the training and test datasets
train_dataset = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11652525.87it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 351643.89it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3211920.61it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7186167.02it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [3]:
def convert_to_list(dataset):
    images = []
    labels = []
    for img, label in dataset:
        # Convert the tensor image to a list
        img_list = img.squeeze().tolist()
        images.append(img_list)
        labels.append(label)
    return images, labels

train_images, train_labels = convert_to_list(train_dataset)
test_images, test_labels = convert_to_list(test_dataset)

print(f"Number of training images: {len(train_images)}")
print(f"Number of training labels: {len(train_labels)}")
print(f"Number of test images: {len(test_images)}")
print(f"Number of test labels: {len(test_labels)}")


Number of training images: 60000
Number of training labels: 60000
Number of test images: 10000
Number of test labels: 10000


In [4]:

train_images_flattened = [[item for sublist in outer for item in sublist] for outer in train_images]
len(train_images_flattened[0])

784

In [5]:
train_labels[219]

5

In [6]:
random.seed(24)

inputs_batch = train_images_flattened
targets_batch = [[l] for l in train_labels]

def one_hot_encode(values, num_classes):
    """
    Convert a list of values into one-hot encoded format.
    Args:
        values (list of list): List containing the values to encode.
        num_classes (int): Number of classes for one-hot encoding.
    Returns:
        list of list: One-hot encoded representation of the input values.
    """
    one_hot_encoded = []
    for value in values:
        one_hot = [0] * num_classes
        one_hot[value[0]] = 1
        one_hot_encoded.append(one_hot)
    return one_hot_encoded

#values = [[1], [0], [2], [3]]
#num_classes = 4

#one_hot_encoded_values = one_hot_encode(values, num_classes)
#print(one_hot_encoded_values)
targets_batch = one_hot_encode(targets_batch, 10)
print(len(targets_batch))





60000


In [11]:
import random

def forward_pass(input_batch, all_neurons_weights, biases):
    batch_output = []
    for inputs in input_batch:
        layer_output = []
        for weights, bias in zip(all_neurons_weights, biases):
            neuron_output = sum(x * w for x, w in zip(inputs, weights)) + bias
            layer_output.append(neuron_output)
        batch_output.append(layer_output)
    return batch_output

def calculate_errors(batch_output_of_forward_pass, target_batch):
    squared_errors = []
    residual_errors = []
    for output, target in zip(batch_output_of_forward_pass, target_batch):
        try:
            sample_squared_errors = [(o - t) ** 2 for o, t in zip(output, target)]
        except OverflowError as e:
            print("Overflow error.", e)
            sample_squared_errors = 0
        #sample_squared_errors = [(o - t) ** 2 for o, t in zip(output, target)]
        sample_residual_errors = [(o - t) for o, t in zip(output, target)]
        squared_errors.append(sample_squared_errors)
        residual_errors.append(sample_residual_errors)
    return squared_errors, residual_errors

def calculate_weight_and_bias_deltas(output_of_calculate_errors, input_batch, learning_rate):
    residual_errors = output_of_calculate_errors[1]
    weight_deltas = [[0 for _ in range(len(input_batch[0]))] for _ in range(len(residual_errors[0]))]
    bias_deltas = [0 for _ in range(len(residual_errors[0]))]

    for residual_error, inputs in zip(residual_errors, input_batch):
        for neuron_index in range(len(residual_error)):
            for feature_index in range(len(inputs)):
                weight_deltas[neuron_index][feature_index] += learning_rate * residual_error[neuron_index] * inputs[feature_index]
            bias_deltas[neuron_index] += learning_rate * residual_error[neuron_index]

    return weight_deltas, bias_deltas

def update_weights_biases(output_of_calculate_weight_and_bias_deltas, weights, biases):
    weight_deltas, bias_deltas = output_of_calculate_weight_and_bias_deltas
    for i in range(len(weights)):
        for j in range(len(weights[i])):
            weights[i][j] -= weight_deltas[i][j]
        biases[i] -= bias_deltas[i]
    return weights, biases

def train(num_epochs, training_samples, target_samples, batch_size, learning_rate):
    num_features = len(training_samples[0])
    num_neurons = len(target_samples[0])

    weights = [[random.random() for _ in range(num_features)] for _ in range(num_neurons)]
    biases = [random.random() for _ in range(num_neurons)]

    for epoch in range(num_epochs):
        for i in range(0, len(training_samples), batch_size):
            input_batch = training_samples[i:i+batch_size]
            target_batch = target_samples[i:i+batch_size]

            batch_output = forward_pass(input_batch, weights, biases)
            errors = calculate_errors(batch_output, target_batch)
            deltas = calculate_weight_and_bias_deltas(errors, input_batch, learning_rate)
            weights, biases = update_weights_biases(deltas, weights, biases)

        if epoch % 10 == 0:
            total_error = sum(sum(e) for e in errors[0])
            print(f"Epoch {epoch}, Error: {total_error}")

    return weights, biases

training_samples = inputs_batch[:500]
target_samples  = targets_batch[:500]
num_epochs = 5
batch_size = 25
learning_rate = 0.01

weights, biases = train(num_epochs, training_samples, target_samples, batch_size, learning_rate)
print("Trained weights:", weights)
print("Trained weights len:", str(len(weights[0])))
print("Trained biases:", biases)
print("Trained biases:", biases)
print("Trained biases len:", str(len(biases)))


Epoch 0, Error: 3.4791639183255476e+40
Trained weights: [[0.9477889863878404, 0.45360684642827664, 0.307304633660494, 0.6762665194647403, 0.6755664115594154, 0.49114556067146464, 0.15535102437170945, 0.02919751764789158, 0.5221555741992665, 0.6927060578733408, 0.4379412656217171, 0.5141748761795447, 0.13969657216010933, 0.14343343325409563, 0.6386329274884958, 0.7419078215137562, 0.9650636953609621, 0.94909942801637, 0.13556534521419417, 0.48510488743220737, 0.762016133535415, 0.8046728807808317, 0.6233750807734354, 0.8051753297516965, 0.2517135527054267, 0.9296311419698299, 0.07607923923615645, 0.7548027794647091, 0.6014943662826782, 0.7893691613244107, 0.28455335186983255, 0.36135808489001775, 0.3553157143867415, 0.5731290666964897, 0.8653372283383725, 0.24371506276906796, 0.1838204379961963, 0.03882544530686338, 0.8141425647076532, 0.14056748124148666, 0.2531263965857389, 3.9822698915674374e+80, 9.802510348827234e+80, -5.502118667710422e+83, -7.34792209024965e+83, -1.844184476812120

In [8]:
[float('inf')] * 10

[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]